# text summarization


In [70]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [71]:
from langchain_groq import ChatGroq

api_key=os.getenv("GROK_API_KEY")

llm = ChatGroq(groq_api_key=api_key , model="gemma2-9b-it")
# llm = ChatGroq(groq_api_key=api_key , model="llama3-8b-8192")

In [72]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EF96636630>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EF9663F020>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [73]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [74]:
speech = """
My dear fellow citizens,

Today, as I stand before you, I feel an immense sense of pride in the strength, unity, and resolve of 140 crore Indians. This land, which has given birth to great saints, warriors, and visionaries, is once again rising with a new energy, a new hope, and a new vision for a better tomorrow.

Brothers and sisters, the 21st century is India's century. This is not a statement of ambition—it is a declaration backed by the hard work of our youth, the innovation of our scientists, the resilience of our farmers, and the determination of every Indian.

From the remotest village to the most advanced digital city, India is changing. We are creating opportunities, empowering dreams, and building a future where every Indian—no matter their background—has a chance to succeed. Whether it's Digital India, Skill India, Make in India, or Startup India—each initiative is a brick in the foundation of a New India.

Friends, we are not just aiming for economic growth, we are building a nation of values, a nation of inclusivity, a nation where development is both sustainable and equitable. We are taking our ancient civilization's wisdom and merging it with the power of modern technology.

Today, India is being recognized globally—not just as the world's largest democracy—but as a torchbearer of peace, progress, and promise. Whether it is our leadership in the fight against climate change, our global vaccine contributions, or our voice for the Global South—we speak not only for ourselves but for the world.

I urge every citizen—let us rise above differences. Let us work shoulder to shoulder. Let us walk together in this journey of Viksit Bharat. Together, we will build an India that future generations will be proud of.

Bharat Mata ki Jai!
Jai Hind!
"""


In [75]:
speech

"\nMy dear fellow citizens,\n\nToday, as I stand before you, I feel an immense sense of pride in the strength, unity, and resolve of 140 crore Indians. This land, which has given birth to great saints, warriors, and visionaries, is once again rising with a new energy, a new hope, and a new vision for a better tomorrow.\n\nBrothers and sisters, the 21st century is India's century. This is not a statement of ambition—it is a declaration backed by the hard work of our youth, the innovation of our scientists, the resilience of our farmers, and the determination of every Indian.\n\nFrom the remotest village to the most advanced digital city, India is changing. We are creating opportunities, empowering dreams, and building a future where every Indian—no matter their background—has a chance to succeed. Whether it's Digital India, Skill India, Make in India, or Startup India—each initiative is a brick in the foundation of a New India.\n\nFriends, we are not just aiming for economic growth, we 

In [76]:
chat_message = [
    SystemMessage(content="You are an expert in summarizing speeches."),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech:\n\nText: {speech}")
]

In [77]:
llm.get_num_tokens(speech)

397

# this is first method  ---->> to summarize  >> the data

In [78]:
llm(chat_message).content

'This speech celebrates India\'s progress and potential. The speaker asserts that the 21st century belongs to India, thanks to the hard work and innovation of its people. \n\nHe highlights various government initiatives aimed at economic growth, digital advancement, and skill development, emphasizing their role in building a "New India."\n\nThe speaker also stresses the nation\'s commitment to inclusivity, sustainability, and global leadership, showcasing India\'s contributions to peace, progress, and global challenges like climate change. \n\nFinally, he calls for national unity and collective effort to build a prosperous and inclusive India for future generations. \n\n\n\n'

# 2-nd method

# prompt template text summarization

In [79]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

generictemplates ="""
write a summary of the following speech:
speech:{speech}
Translate the summary into {language}.

"""
# Define the language to translate into
prompt = PromptTemplate(
    input_variables=["speech", "language"],
    template=generictemplates
)

prompt


PromptTemplate(input_variables=['language', 'speech'], input_types={}, partial_variables={}, template='\nwrite a summary of the following speech:\nspeech:{speech}\nTranslate the summary into {language}.\n\n')

In [80]:
complete_prompt=prompt.format(speech=speech,language="French")

complete_prompt

"\nwrite a summary of the following speech:\nspeech:\nMy dear fellow citizens,\n\nToday, as I stand before you, I feel an immense sense of pride in the strength, unity, and resolve of 140 crore Indians. This land, which has given birth to great saints, warriors, and visionaries, is once again rising with a new energy, a new hope, and a new vision for a better tomorrow.\n\nBrothers and sisters, the 21st century is India's century. This is not a statement of ambition—it is a declaration backed by the hard work of our youth, the innovation of our scientists, the resilience of our farmers, and the determination of every Indian.\n\nFrom the remotest village to the most advanced digital city, India is changing. We are creating opportunities, empowering dreams, and building a future where every Indian—no matter their background—has a chance to succeed. Whether it's Digital India, Skill India, Make in India, or Startup India—each initiative is a brick in the foundation of a New India.\n\nFrien

In [81]:
llm.get_num_tokens(complete_prompt)

418

In [82]:
llm_chain = LLMChain(llm=llm,prompt=prompt)

summary = llm_chain.run({'speech':speech,'language':"Hindi"})

summary

"## Speech Summary:\n\nThe speaker expresses immense pride in the unity and strength of India.  He declares the 21st century as India's century, citing the hard work, innovation, and determination of its people. He highlights various initiatives like Digital India, Skill India, Make in India, and Startup India as building blocks for a New India. \n\nThe speaker emphasizes that India's growth isn't just about economics, but also about building a nation of values, inclusivity, and sustainability. He acknowledges India's global recognition as a leader in peace, progress, and promise, citing its contributions in fighting climate change, providing vaccines, and advocating for the Global South. \n\nFinally, he calls upon every citizen to unite and work together towards building a prosperous India for future generations.\n\n\n## Hindi Translation:\n\n**वक्तव्य सारांश:**\n\nवक्ता भारत की एकता और शक्ति में गर्व व्यक्त करता है। वह 21वीं सदी को भारत का कहता है, लोगों के कड़ी मेहनत, नवाचार और दृढ़

# Stuff-document-chain  Text Summarization

# it is for small dataset 

In [83]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()

docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

# create prompt template

In [84]:
template = """
write a concise and short summary of the following speech ,
speech :{text}
"""

prompt = PromptTemplate(input_variable=["text"], template=template)

prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nwrite a concise and short summary of the following speech ,\nspeech :{text}\n')

In [85]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm , chain_type="stuff" , prompt=prompt , verbose=True)

output_summary =chain.run(docs)

output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

write a concise and short summary of the following speech ,
speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online i

"President A.P.J. Abdul Kalam's departing speech highlights his vision for a developed India by 2020. He emphasizes the importance of:\n\n* **Youth empowerment:** He sees the 540 million youth as India's greatest asset and urges them to be driven by a desire to contribute to the nation's progress.\n\n* **Rural development:**  He stresses the need to empower villages, improve infrastructure, and bridge the urban-rural divide.\n\n* **Technological advancement:** Kalam highlights the role of technology in driving economic growth and societal transformation, citing initiatives like the Pan African e-Network.\n\n* **National unity and defense:**  He expresses pride in India's armed forces and emphasizes the importance of a strong and united nation.\n\n* **Sustainable development:**  He advocates for a balanced approach to development that considers both economic growth and environmental protection.\n\n\nHe shares several inspiring anecdotes illustrating these points, showcasing the resilien

# 4th method

# Map reduce to Summarization Large Documents

# for large text 

In [86]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [87]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [88]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [89]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nwrite a concise and short summary of the following speech ,\nspeech :{text}\n')

In [90]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [91]:
final_prompt='''
Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a Motivation Title,Start the precise summary with an introduction and provide the summary in number \npoints for the speech.\nSpeech:{text}\n\n')

In [92]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have m

'##  A Nation United: Dr. Kalam\'s Vision for a Developed India\n\nThis speech is a powerful reflection on Dr. Kalam\'s five years as President of India, filled with hope, unity, and a clear vision for the nation\'s future. \n\nHere are the key takeaways:\n\n1. **Accelerated Development Driven by Youth:** Dr. Kalam emphasizes the crucial role of India\'s youth in propelling the nation towards development. He acknowledges their aspirations for a developed nation by 2020 and pledges to work towards fulfilling those dreams.\n2. **Empowering Rural India:**  Dr. Kalam highlights the vital role of villages in national progress. He advocates for empowering village councils, improving connectivity, and leveraging rural skills for economic growth, citing the Periyar PURA project as a successful model.\n3. **Second Green Revolution:** The speaker expresses confidence in India\'s ability to achieve a "second green revolution," doubling agricultural production while conserving resources through co

# Refine chain summarization

In [93]:
chain=load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)
output_summary=chain.run(final_documents)
output_summary



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'In his departing speech, A P J Abdul Kalam reflects on his five years as President of India, expressing gratitude for the diverse interactions he\'s had with people from all walks of life. He highlights ten key messages from his experiences, emphasizing the importance of:\n\n* **Youth aspirations** driving development. He recounts a young girl\'s question challenging him to make India developed by 2020, emphasizing the deep desire for progress among India\'s youth. He sees this aspiration echoed in the millions of young people he\'s met and reaffirms his commitment to fulfilling their dream, mentioning the Lead India 2020 Foundation as an example of youth-led initiatives.  Kalam shares specific examples of the positive impact of the Lead India 2020 Foundation, particularly in Medak district of Andhra Pradesh, citing the transformation of students in terms of self-discipline, love for their family and teachers, and a sense of duty towards the nation.  \n* **Empowering villages** throug